In [16]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.redis import RedisSaver  
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from typing import TypedDict, Annotated, List
from dotenv import load_dotenv
import redis

##################################################################################################
# REDIS SECTION
r = redis.Redis(host="localhost", port=6379, decode_responses=True)
##################################################################################################
# LLM Section
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
##################################################################################################
# Langgraph Section
class ChatState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

# Define nodes
def chat_node(state: ChatState):
    return {
        'messages': llm.invoke(state['messages']) 
    }

checkpointer = RedisSaver(
    redis_url="redis://localhost:6379",  
)
# Define graph
graph = StateGraph(ChatState)

# Add nodes
graph.add_node("chat_node", chat_node)

# Add edges
graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

# Compile chatbot
chatbot = graph.compile(checkpointer=checkpointer)

##################################################################################################
# Inference Section 

config = {
        "configurable": {
            "thread_id": 1
        }
    }

def respond(user_input):
    #print(chatbot.checkpointer)
    return chatbot.invoke(
        {'messages': [HumanMessage(content=user_input)]},
        config=config
    )['messages'][-1].content



In [20]:
list(chatbot.get_state_history(config))


RedisSearchError: Error while searching: No such index checkpoints

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import Annotated
from typing_extensions import TypedDict
from operator import add

class State(TypedDict):
    foo: str
    bar: Annotated[list[str], add]

def node_a(state: State):
    return {"foo": "a", "bar": ["a"]}

def node_b(state: State):
    return {"foo": "b", "bar": ["b"]}


workflow = StateGraph(State)
workflow.add_node(node_a)
workflow.add_node(node_b)
workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", END)

checkpointer = InMemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "1"}}
graph.invoke({"foo": ""}, config)

{'foo': 'b', 'bar': ['a', 'b']}

In [23]:
graph.get_state(config)


StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08ca1b-a62c-666e-8002-019977f3587e'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-08T10:51:09.314409+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08ca1b-a62b-6113-8001-45c1b9d7aa54'}}, tasks=(), interrupts=())